In [1]:
import pandas as pd
import numpy as np


chrom = 22
df = pd.read_pickle('chrom_%d.pkl' % chrom)
df.set_index(['gene', 'intron'], inplace=True)
df.sort_index(inplace=True)
df.head()

chrom  lmm-pval  lmm-rank-pval       pos  qep-pval  \
gene            intron                                                       
ENSG00000025770 1          22  0.613807       0.480221  50896901  0.389656   
                1          22  0.316533       0.574893  50897215  0.533459   
                1          22  0.043711       0.138918  50897314  0.099098   
                1          22  0.514677       0.750123  50897692  0.828003   
                1          22  0.000753       0.021789  50897868  0.007409   

                                 snp_id  
gene            intron                   
ENSG00000025770 1       snp_22_50896901  
                1       snp_22_50897215  
                1       snp_22_50897314  
                1       snp_22_50897692  
                1       snp_22_50897868

# Bonferroni correction

In [2]:
ntests = len(df)
print("Number of tests: %d" % ntests)
df['lmm-pval'] = np.clip(df['lmm-pval'] * ntests, 0, 1)
df['lmm-rank-pval'] = np.clip(df['lmm-rank-pval'] * ntests, 0, 1)
df['qep-pval'] = np.clip(df['qep-pval'] * ntests, 0, 1)

Number of tests: 137228


In [3]:
from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.charts import BoxPlot, Bar
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [4]:
def plot_curve(perc=False, exclude=False, lthr=-5, rthr=-0.30103):

    data = pd.DataFrame(columns=('pval', 'method', 'color'))

    def append_df(data, df, name):
        N = len(df['%s-pval' % name])
        # BONFERRONI ACROSS THE ENTIRE SET OF GENE-INTRON PAIRS
        d = [np.asarray(df['%s-pval' % name]), [name] * N]
        data = pd.concat([data, pd.DataFrame({'pval': d[0], 'method': d[1]})])
        return data

    data = append_df(data, df, 'lmm')
    data = append_df(data, df, 'lmm-rank')
    data = append_df(data, df, 'qep')
    
    nhits = {'lmm-rank':[], 'qep':[]}
    if not exclude:
        nhits['lmm'] = []
    
    pts = np.logspace(lthr, rthr)
    for pt in pts:

        if perc:
            grouped = data.groupby(data['method']).agg({'method': lambda x: x[0], 'pval': lambda x: 100*sum(x<=pt)/float(len(x))})
        else:
            grouped = data.groupby(data['method']).agg({'method': lambda x: x[0], 'pval': lambda x: sum(x<=pt)})


        nhits['lmm-rank'].append(grouped.loc['lmm-rank', 'pval'])
        if not exclude:
            nhits['lmm'].append(grouped.loc['lmm', 'pval'])
        nhits['qep'].append(grouped.loc['qep', 'pval'])

    p = figure(title="Power", tools=['save,reset'])
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = 'P-value threshold'
    if perc:
        p.yaxis.axis_label = 'Percentage of hits'
    else:
        p.yaxis.axis_label = 'Number of hits'
    p.line(pts, nhits['lmm-rank'], color='green', legend='lmm-rank')
    if not exclude:
        p.line(pts, nhits['lmm'], color='blue', legend='lmm')
    p.line(pts, nhits['qep'], color='red', legend='qep')
    p.legend.location = "bottom_right"
    return p

In [5]:
p1 = plot_curve(True)
p2 = plot_curve(True, True)
p3 = plot_curve(False)
p4 = plot_curve(False, True)
show(gridplot([[p1,p2],[p3,p4]], plot_width=400, plot_height=400))

In [6]:
lthr = -5
rthr = -1
p1 = plot_curve(True, lthr=lthr, rthr=rthr)
p2 = plot_curve(True, True, lthr=lthr, rthr=rthr)
p3 = plot_curve(False, lthr=lthr, rthr=rthr)
p4 = plot_curve(False, True, lthr=lthr, rthr=rthr)
show(gridplot([[p1,p2],[p3,p4]], plot_width=400, plot_height=400))

In [7]:
lthr = -10
rthr = -2
p1 = plot_curve(True, lthr=lthr, rthr=rthr)
p2 = plot_curve(True, True, lthr=lthr, rthr=rthr)
p3 = plot_curve(False, lthr=lthr, rthr=rthr)
p4 = plot_curve(False, True, lthr=lthr, rthr=rthr)
show(gridplot([[p1,p2],[p3,p4]], plot_width=400, plot_height=400))